In [1]:
import os
from dotenv import load_dotenv

# Set environment variables in the .env file.
load_dotenv()

OPENAI_API_TYPE = os.environ["OPENAI_API_TYPE"]
OPENAI_API_VERSION = os.environ["OPENAI_API_VERSION"]
OPENAI_API_BASE = os.environ["OPENAI_API_BASE"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [73]:
import openai

openai.api_type = OPENAI_API_TYPE
openai.api_key = OPENAI_API_KEY

openai.base_url = OPENAI_API_BASE
openai.api_version = OPENAI_API_VERSION

system_prompt = """Assistant acts as a personal assistant for employees of RazType.
- You are provided with a list of functions. If there is a relevant function, prioritize the using that function.
- If a function is to be called, do not make assumptions on parameter inputs with the user. Ask the user for the parameters.

Example:
User: Can you help me generate a shortened URL for https://www.raztype.com?
Assistant: What path would you like to use for the shortened URL?"""

functions = [
    {
        "name": "generate_shortened_url",
        "description": "Generates a shortened URL from a long URL. Outputs the shortened URL based on the path input.",
        "parameters": {
            "type": "object",
            "properties": {
                "long_url": {
                    "type": "string",
                    "description": "The long URL to shorten."
                },
                "short_path": {
                    "type": "string",
                    "description": "The path to use for the shortened URL."
                }
            },
            "required": ["long_url", "short_path"]
        }
    },
    {
        "name": "add_task",
        "description": "Adds a task to the user's to-do list.",
        "parameters": {
            "type": "object",
            "properties": {
                "task": {
                    "type": "string",
                    "description": "The task to add to the to-do list."
                }
            },
            "required": ["task"]
        }
    },
    {
        "name": "send_kudos_email",
        "description": "Sends an appreciation message to a colleague, where the colleague's manager is on the CC list.",
        "parameters": {
            "type": "object",
            "properties": {
                "colleague_email": {
                    "type": "string",
                    "description": "The e-mail address of the colleague."
                },
                "kudos_message": {
                    "type": "string",
                    "description": "The appreciation message."
                }
            },
            "required": ["colleague_email", "kudos_message"]
        }
    }
]

In [74]:
response = openai.ChatCompletion.create(
    engine="gpt4",
    temperature=0,
    functions=functions,
    function_call="auto",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "What is the link to the Microsoft website?"}
    ]
)

print(response["choices"][0]["message"])

{
  "role": "assistant",
  "content": "The link to the Microsoft website is https://www.microsoft.com."
}


In [76]:
response = openai.ChatCompletion.create(
    engine="gpt4",
    temperature=0,
    functions=functions,
    function_call="auto",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "What is the link to the Microsoft website?"},
        {"role": "assistant", "content": "The link to the Microsoft website is: https://www.microsoft.com/"},
        {"role": "user", "content": "Can you help me generate a shortened URL for the Microsoft website?"}
    ]
)

print(response["choices"][0]["message"])

{
  "role": "assistant",
  "content": "What path would you like to use for the shortened URL?"
}


In [77]:
response = openai.ChatCompletion.create(
    engine="gpt4",
    temperature=0,
    functions=functions,
    function_call="auto",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "What is the link to the Microsoft website?"},
        {"role": "assistant", "content": "The link to the Microsoft website is: https://www.microsoft.com/"},
        {"role": "user", "content": "Can you help me generate a shortened URL for the Microsoft website?"},
        {"role": "assistant", "content": "Sure! I can help you with that. What path would you like to use for the shortened URL?"},
        {"role": "user", "content": "ms"}
    ]
)

print(response["choices"][0]["message"])

{
  "role": "assistant",
  "function_call": {
    "name": "generate_shortened_url",
    "arguments": "{\n  \"long_url\": \"https://www.microsoft.com/\",\n  \"short_path\": \"ms\"\n}"
  }
}


In [78]:
response = openai.ChatCompletion.create(
    engine="gpt4",
    temperature=0,
    functions=functions,
    function_call="auto",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "What functions can you help me with?"}
    ]
)

print(response["choices"][0]["message"])

{
  "role": "assistant",
  "content": "I can assist you with the following functions:\n\n1. `generate_shortened_url`: This function generates a shortened URL from a long URL. You need to provide the long URL and the path you want to use for the shortened URL.\n\n2. `add_task`: This function adds a task to your to-do list. You just need to provide the task you want to add.\n\n3. `send_kudos_email`: This function sends an appreciation message to a colleague, where the colleague's manager is on the CC list. You need to provide the email address of your colleague and the appreciation message you want to send."
}


In [85]:
# Finding: only suggests ONE function to call
response = openai.ChatCompletion.create(
    engine="gpt4",
    temperature=0,
    functions=functions,
    function_call="auto",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "What is the link to the Microsoft website?"},
        {"role": "assistant", "content": "The link to the Microsoft website is: https://www.microsoft.com/"},
        {"role": "user", "content": "Can you help me generate a shortened URL named 'ms' and send a kudos to bob@contoso.com?"}
    ]
)

print(response["choices"][0]["message"])

{
  "role": "assistant",
  "function_call": {
    "name": "generate_shortened_url",
    "arguments": "{\n  \"long_url\": \"https://www.microsoft.com/\",\n  \"short_path\": \"ms\"\n}"
  }
}
